In [1]:
url="https://www.amazon.in/soundcore-Bluetooth-Headphones-Cancelling-Personalization/dp/B0C3HCD34R/ref=sr_1_2_sspa?crid=2OBR9MH0Q68TM&dib=eyJ2IjoiMSJ9.SxNPJoR7Jp07YgVlcCqtQQSkyDXEFbWC06vmworvSqkoynXhinkdZBL21YL8-6EZV6UIUvWgMst1-wAwMddo0RZNdQUKYeIEYJnmPUb3CuhMoKsZSTZvlqhMj14ET4EQ7mJk6WY7s_SSKP5MqjfRIRgpldYRa95oFiwk95-mvY3hIDBKgdUgF6ZELqru0icBLZ4thTBXnn8wwhGXJhtAJWvEbtHJvj07JYq6Vy3wj0M.HlXWVJuXhOlbOdhC1SQqM5NALvGsoC9CzQeRtmiouYc&dib_tag=se&keywords=headphones&qid=1758086366&sprefix=%2Caps%2C282&sr=8-2-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1"

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

options = Options()
# Run in visible mode (non-headless)
service = Service("C:/Program Files/chromedriver-win64/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

# Load the page
driver.get(url)  # your product URL

# Wait for dynamic content to load
import time
time.sleep(30)  # or use WebDriverWait for better control

# Get the full HTML after JavaScript has rendered
html = driver.page_source
#print(html)

driver.quit()


In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, "html.parser")
title = soup.find(id="productTitle")
if title:
    print("Product Title:", title.get_text(strip=True))
else:
    print("Product title not found.")

TITLE=title.get_text(strip=True)
#print(t)

Product Title: soundcore by Anker Q20i Wireless Bluetooth Over-Ear Headphones with Hybrid Active Noise Cancelling, 40h Playtime in ANC Mode, Hi-Res Audio, Deep Bass, Personalization via App (Black)


In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "html.parser")

price_tag = soup.find("span", class_="a-price a-text-price")  # fixed class name
if price_tag:
    price_value = price_tag.find("span", class_="a-offscreen")
    if price_value:
        price = price_value.get_text(strip=True)
        print("Price:", price)  # Output: Price: ₹699
    else:
        print("Price not found inside a-price.")
else:
    print("Price container not found.")

MRP=price

Price: ₹9,598


In [5]:
discount_tag = soup.find("span", class_="a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage")
if discount_tag:
    discount = discount_tag.get_text(strip=True).replace("-", "").replace("%", "")
    print("Discount:", discount)  # Output: Discount: 43
else:
    print("Discount not found.")

DISCOUNT=discount

Discount: 59


In [6]:
price_tag = soup.find("span", class_="a-price")
if price_tag:
    symbol = price_tag.find("span", class_="a-price-symbol")
    whole = price_tag.find("span", class_="a-price-whole")
    
    if symbol and whole:
        full_price = symbol.get_text(strip=True) + whole.get_text(strip=True)
        print("Price after Discount:", full_price)
    else:
        print("Price components not found.")
else:
    print("Price tag not found.")

PRICE=whole.get_text(strip=True)

Price after Discount: ₹3,899.


In [7]:
import pandas as pd

pd.set_option('display.max_colwidth', None)  # Show full column width
pd.set_option('display.width', 1000)         # Set wide display width

data = {
    "Title": [TITLE],
    "MRP": [MRP],
    "Discount (%)": [DISCOUNT],
    "Price After Discount": [PRICE]
}
df=pd.DataFrame(data)


print(df.to_string(index=False))  # Prevent wrapping and show full table

                                                                                                                                                                                 Title    MRP Discount (%) Price After Discount
soundcore by Anker Q20i Wireless Bluetooth Over-Ear Headphones with Hybrid Active Noise Cancelling, 40h Playtime in ANC Mode, Hi-Res Audio, Deep Bass, Personalization via App (Black) ₹9,598           59               3,899.


In [8]:
import pandas as pd
from IPython.display import display, HTML

data = {
    "Title": [TITLE],           # Full title
    "MRP": [MRP],               # Might be "₹699" or 699
    "Discount (%)": [DISCOUNT], # e.g. 43
    "Price After Discount": [PRICE]  # Might be "399.0"
}

df = pd.DataFrame(data)

# Clean MRP & Price — remove ₹ if present and convert to numbers
def clean_number(x):
    if x is None or str(x).strip() == "":
        return None
    try:
        return float(str(x).replace("₹", "").replace(",", "").strip())
    except:
        return None


df["MRP"] = df["MRP"].apply(clean_number)
df["Price After Discount"] = df["Price After Discount"].apply(clean_number)

# Format with ₹
df["MRP"] = df["MRP"].apply(lambda x: f"₹{int(x)}" if pd.notnull(x) else "N/A")
df["Price After Discount"] = df["Price After Discount"].apply(
    lambda x: f"₹{int(x)}" if pd.notnull(x) and x.is_integer() else f"₹{x:.2f}" if pd.notnull(x) else "N/A"
)

# Style: center Title column text
styled_table = df.style.set_table_styles([
    {"selector": "th", "props": [("text-align", "center")]},  # center headers
    {"selector": "td.col0", "props": [("text-align", "center")]}  # center first column (Title)
])

display(HTML(styled_table.to_html()))


,Title,MRP,Discount (%),Price After Discount
0,"soundcore by Anker Q20i Wireless Bluetooth Over-Ear Headphones with Hybrid Active Noise Cancelling, 40h Playtime in ANC Mode, Hi-Res Audio, Deep Bass, Personalization via App (Black)",₹9598,59,₹3899
